In [6]:
!pip install transformers>=4.39.0 -qU

In [2]:
!pip install git+https://github.com/jiaweizzhao/GaLore -q

  Preparing metadata (setup.py) ... done


In [4]:
!pip install datasets trl -qU

In [5]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-aaff7c88-a5b1-e742-8c16-aceda444a2cd)


In [7]:
import torch
import datasets
from transformers import TrainingArguments, AutoConfig, AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM


train_dataset = datasets.load_dataset('Cognitive-Lab/Kannada-Instruct-dataset', split='train')

args = TrainingArguments(
    output_dir="./test-galore",
    max_steps=100,
    per_device_train_batch_size=2,
    optim="galore_adamw",
    optim_target_modules=["attn", "mlp"]
)

model_id = "google/gemma-2b-it"

config = AutoConfig.from_pretrained(model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_config(config).to(0)

def formatting_prompts_func(kannada):
    output_texts = []
    for i in range(len(kannada['features'])):
        text = f"### Question: {kannada['original_instruction'][i]}\n ### Answer: {kannada['original_output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    dataset_text_field='translated_output',
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    max_seq_length=512,
)

trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.95 GiB. GPU 0 has a total capacity of 15.89 GiB of which 94.12 MiB is free. Process 1307 has 15.80 GiB memory in use. Of the allocated memory 15.55 GiB is allocated by PyTorch, and 1.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)